# Crashes to JSON

Use `Data Query Tool` to create a new export from UCONN Crashes (Town=Hartford, last five years): https://www.ctcrash.uconn.edu/QueryTool2.action

* Since the tool does not let you export large datasets (over 90,000 persons), export multiple times for different time periods and specify all export IDs in the `exports` list below.

This notebook combines data from two tables (0 and 2) into a small JSON file and a CSV file (same content, different formats), which is an array of objects that look like this:
```js
{
    "id": 770826,    // crash id (used to pull the diagram)
    "x": 41.73936,   // latitude
    "y": -72.66404,  // longitude
    "d": 1593820800, // timestamp
    "p": 0,          // involves a pedestrian? (1=true, 0=false)
    "c": 0,          // involves a cyclist? (1=true, 0=false)
    "r": 1,          // route class (1=Interstate, 2=USRoute, 3=State, 4=Local)
    "s": 0,          // crash severity (O=no injuries, A=any injury, K=fatal)
}
```

Note: the timestamp `d` needs to be multiplied by `100,000` for a proper JS timestamp.

In [1]:
import pandas as pd

In [2]:
exports = ['166508']

### Import roles from table 2

In [3]:
roles = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_2.csv',
        skiprows=1,
        encoding='utf-8',
        usecols=['CrashId', 'Person Type Text Format']
    )
    roles.append( df )

roles_df = pd.concat(roles).groupby(['CrashId'])['Person Type Text Format'].unique() # .groupby(['CrashId'])['Person Type Text Format'].unique()


### Import crashes from table 0

In [4]:
localData = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_0.csv',
        skiprows=1,
        usecols=['CrashId', 'Latitude', 'Longitude', 'Date Of Crash',
                 'Time of Crash', 'Crash Severity', 'Route Class',
                 'Roadway Name', 'Intersecting Roadway Name',
                 'Town Name']
    )
    localData.append( df )

localData_df = pd.concat(localData)

crashesTable0_df = localData_df.merge(roles_df, on='CrashId', how='inner')

crashesTable0_df.head()

,CrashId,Latitude,Longitude,Town Name,Date Of Crash,Time of Crash,Crash Severity,Route Class,Roadway Name,Intersecting Roadway Name,Person Type Text Format
0,12349,41.839598,-71.877738,Killingly,2015-04-22,16:47:00,A,1.0,I 395 N,NB EXIT TO RTE 101(HTFD PIKE)(052) : Exit 93,"[Driver, Bicyclist]"
1,16244,41.971878,-72.048918,Woodstock,2015-04-06,06:11:00,A,3.0,RT 197,NaN,"[Driver, Pedestrian, Witness]"
2,21512,41.737468,-72.188262,Windham,2015-01-16,15:07:00,A,4.0,Circle Dr,STONEGATE RD,"[Pedestrian, Driver, Passenger]"
3,21866,41.657526,-72.033100,Canterbury,2015-02-18,18:28:00,K,4.0,Water St,NaN,"[Pedestrian, Witness, Driver]"
4,27582,41.953063,-72.307495,Stafford,2015-01-20,18:54:00,A,3.0,Main St,RTE 140(TOLLAND AVE),"[Pedestrian, Driver]"


Import crashes from table 1


In [5]:
vehicleData = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_1.csv',
        skiprows = 1,
        usecols=['CrashId', 'Direction of Travel Before Crash', 'Contributing Circumstances, Motor Vehicle', 
                 'Posted/Statutory Speed Limit', 'Bike Lanes/Sharrows Present', 'Hit And Run Status']
    )
    vehicleData.append ( df )

crashesTable1_df = pd.concat(vehicleData)

crashesTableFinal_df = crashesTable0_df.merge(crashesTable1_df, on='CrashId', how='inner')

### Merge data and save to .json and .csv

In [6]:
crashesTableFinal_df.to_csv('crashes.csv', index=False)

crashesTableFinal_df.head()


,CrashId,Latitude,Longitude,Town Name,Date Of Crash,Time of Crash,Crash Severity,Route Class,Roadway Name,Intersecting Roadway Name,Person Type Text Format,Direction of Travel Before Crash,"Contributing Circumstances, Motor Vehicle",Posted/Statutory Speed Limit,Hit And Run Status,Bike Lanes/Sharrows Present
0,12349,41.839598,-71.877738,Killingly,2015-04-22,16:47:00,A,1.0,I 395 N,NB EXIT TO RTE 101(HTFD PIKE)(052) : Exit 93,"[Driver, Bicyclist]",N,0.0,35.0,False,False
1,12349,41.839598,-71.877738,Killingly,2015-04-22,16:47:00,A,1.0,I 395 N,NB EXIT TO RTE 101(HTFD PIKE)(052) : Exit 93,"[Driver, Bicyclist]",W,0.0,88.0,False,False
2,16244,41.971878,-72.048918,Woodstock,2015-04-06,06:11:00,A,3.0,RT 197,NaN,"[Driver, Pedestrian, Witness]",W,0.0,45.0,False,False
3,21512,41.737468,-72.188262,Windham,2015-01-16,15:07:00,A,4.0,Circle Dr,STONEGATE RD,"[Pedestrian, Driver, Passenger]",W,97.0,15.0,NaN,NaN
4,21866,41.657526,-72.033100,Canterbury,2015-02-18,18:28:00,K,4.0,Water St,NaN,"[Pedestrian, Witness, Driver]",S,99.0,30.0,False,False


In [7]:
# Rename columns to make JSON output smaller
crashesTableFinal_df.columns = ['id', 'x', 'y', 'tn', 'd', 't', 's', 'r', 'o', 'ir',\
    'pt', 'dt', 'm', 'sl', 'hr', 'blp']

# id = CrashId
# x = Longitude
# y = Latitude
# tn = Town Name
# d = Date of Crash
# t = Time of Crash
# s = Crash Severity
# r = Route Class
# ir = Intersecting Roadway Name
# o = Roadway Name
# pt = Person Type
# dt = Direction of Travel
# m = Contributing Circumstances
# sl = Speed Limit
# hr = Hit and Run
# blp = Bike Lanes Present

crashesTableFinal_df.head()

,id,x,y,tn,d,t,s,r,o,ir,pt,dt,m,sl,hr,blp
0,12349,41.839598,-71.877738,Killingly,2015-04-22,16:47:00,A,1.0,I 395 N,NB EXIT TO RTE 101(HTFD PIKE)(052) : Exit 93,"[Driver, Bicyclist]",N,0.0,35.0,False,False
1,12349,41.839598,-71.877738,Killingly,2015-04-22,16:47:00,A,1.0,I 395 N,NB EXIT TO RTE 101(HTFD PIKE)(052) : Exit 93,"[Driver, Bicyclist]",W,0.0,88.0,False,False
2,16244,41.971878,-72.048918,Woodstock,2015-04-06,06:11:00,A,3.0,RT 197,NaN,"[Driver, Pedestrian, Witness]",W,0.0,45.0,False,False
3,21512,41.737468,-72.188262,Windham,2015-01-16,15:07:00,A,4.0,Circle Dr,STONEGATE RD,"[Pedestrian, Driver, Passenger]",W,97.0,15.0,NaN,NaN
4,21866,41.657526,-72.033100,Canterbury,2015-02-18,18:28:00,K,4.0,Water St,NaN,"[Pedestrian, Witness, Driver]",S,99.0,30.0,False,False


In [8]:
# Transform timestamps
crashesTableFinal_df.d = crashesTableFinal_df.d.apply(lambda x: int(pd.to_datetime(x).timestamp() / 100))
crashesTableFinal_df.t = crashesTableFinal_df.t.apply(lambda x: x[:-3])

# Set up pedestrian and cyclist flags
crashesTableFinal_df['p'] = crashesTableFinal_df.id.apply(lambda x: 1 if x in roles_df and 'Pedestrian' in roles_df[x] else 0)
crashesTableFinal_df['c'] = crashesTableFinal_df.id.apply(lambda x: 1 if x in roles_df and 'Bicyclist' in roles_df[x] else 0)

crashesTableFinal_df.head()

,id,x,y,tn,d,t,s,r,o,ir,pt,dt,m,sl,hr,blp,p,c
0,12349,41.839598,-71.877738,Killingly,14296608,16:47,A,1.0,I 395 N,NB EXIT TO RTE 101(HTFD PIKE)(052) : Exit 93,"[Driver, Bicyclist]",N,0.0,35.0,False,False,0,1
1,12349,41.839598,-71.877738,Killingly,14296608,16:47,A,1.0,I 395 N,NB EXIT TO RTE 101(HTFD PIKE)(052) : Exit 93,"[Driver, Bicyclist]",W,0.0,88.0,False,False,0,1
2,16244,41.971878,-72.048918,Woodstock,14282784,06:11,A,3.0,RT 197,NaN,"[Driver, Pedestrian, Witness]",W,0.0,45.0,False,False,1,0
3,21512,41.737468,-72.188262,Windham,14213664,15:07,A,4.0,Circle Dr,STONEGATE RD,"[Pedestrian, Driver, Passenger]",W,97.0,15.0,NaN,NaN,1,0
4,21866,41.657526,-72.033100,Canterbury,14242176,18:28,K,4.0,Water St,NaN,"[Pedestrian, Witness, Driver]",S,99.0,30.0,False,False,1,0


In [9]:
# Save as JSON
crashesTableFinal_df.to_json('./crashes.json', orient='records', double_precision=5)

# Save as CSV
crashesTableFinal_df.to_csv('crashes.csv', index=False)